In [2]:
import os, sys
import traceback
import pandas as pd
import re
import glob
import time
from string import ascii_uppercase
from natsort import natsorted
import math


'''
This code reads the IDT file generated by Primer and Guide design's script. 
The IDT files obtained from this script have following columns:
 **Name, Seqeunce, Scale, Purification, Ta, Extension Time, Template Length, Template_Name**

This script generatess multiple CSV output in a folder called "Generated_files".

The IDT ordering format are written to a different folder called "IDT_Ordering"

At last, the script Verify that all original  primers and guides are present in IDT order.  
'''

print('\t',os.getcwd(), '\n')

def write_to_subfolder(result_df, new_file, subdirectory):
    
    curr_dir = os.getcwd()
    #subdirectory = 'Generated_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = True)
        os.chdir('..')
        
def write_to_subfolder_excel(result_df, new_file, subdirectory):
    
    new_file = new_file[:-4]+'.xlsx'
    curr_dir = os.getcwd()
    #subdirectory = 'Generated_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_excel(new_file, index = False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_excel(new_file, index = True)
        os.chdir('..')

        
        
        
def Create_Output():

    df_all = pd.DataFrame()
    df_temp = pd.DataFrame()

    filenames = glob.glob("Primers_Guides_neb*.csv")
    filenames = natsorted(filenames)
    Lengths = []

    for file in filenames:
        print("File read:-  ", file)
        df_temp = pd.read_csv(file)
        Lengths.append(len(df_temp))
        df_all = df_all.append(df_temp)
    
    print('\n These two must be equal:  ')
    print(" \t Total number of primers and guides:-  ", sum(Lengths), )
    print(" \t Number after combining all files:-  ", len(df_all), '\n')
            
    df_all  = df_all.drop(['Scale', 'Purification', 'Extension Time'], axis=1)
    df_Primers_IDT, df_Guides_IDT = Separate_primers_guides(df_all, filenames)
    
    compare_seqeunces(filenames, df_Primers_IDT, df_Guides_IDT)
    
        
def Separate_primers_guides (df_all, filenames):

    df_all  = df_all.astype(str)
    df_all  = df_all.reset_index(drop=True)
    write_to_subfolder(df_all, 'Primers_guides_all_combined.csv', 'Generated_files')
    
    df_Primers = pd.DataFrame()
    df_Guides = pd.DataFrame()

    for index, row in df_all.iterrows():
        Name = df_all['Name'].loc[index]
        isGuide = re.search("[G][0-9]\S[j][0-9]", Name)
        isPrimer = ((re.search("[F][P]", Name) or re.search("[R][P]", Name)))            

        if isGuide:
            df_Guides = df_Guides.append(df_all.loc[index])[df_all.columns.tolist()]
        elif isPrimer:
            df_Primers = df_Primers.append(df_all.loc[index])[df_all.columns.tolist()]

    df_Primers  = df_Primers.reset_index(drop=True)
    df_Guides = df_Guides.reset_index(drop=True)
    df_Primers_IDT, df_Guides_IDT = Unique_Primers_Guides(df_all, df_Primers, df_Guides)
    
    return df_Primers_IDT, df_Guides_IDT

    
def Unique_Primers_Guides(df_all, df_Primers, df_Guides):
    
    df_unique = df_all.drop_duplicates(subset=['Sequence'])
    df_unique  = df_unique.reset_index(drop=True)
    write_to_subfolder(df_unique, 'Primers_guides_unique.csv', 'Generated_files')
    
    try:
        df_Primers_IDT  = df_Primers.drop(['Ta', 'Template Length', 'Template_Name'], axis=1)
    except:
        df_Primers_IDT  = df_Primers.drop(['Ta', 'Template Length'], axis=1)
        print('\n \t Need to add Template name and re-run the script \n')
    
    
    df_Primers_IDT = df_Primers_IDT.drop_duplicates(subset=['Sequence'])
    df_Primers_IDT  = df_Primers_IDT.reset_index(drop=True)
    write_to_subfolder(df_Primers_IDT, 'Primers_for_IDT_order.csv', 'Generated_files')
    
    try:
        df_Guides_IDT  = df_Guides.drop(['Ta', 'Template Length', 'Template_Name'], axis=1)
    except:
        df_Guides_IDT  = df_Guides.drop(['Ta', 'Template Length'], axis=1)
        print('\n \t Need to add Template name and re-run the script \n')


    
    df_Guides_IDT = df_Guides_IDT.drop_duplicates(subset=['Sequence'])
    df_Guides_IDT  = df_Guides_IDT.reset_index(drop=True)
    write_to_subfolder(df_Guides_IDT, 'Guides_for_IDT_order.csv', 'Generated_files') 
    
    print('\n  Writing 96 well IDT positions \n')

    write_96well_IDT_files(df_Primers_IDT, 'Primers')
    write_96well_IDT_files(df_Guides_IDT, 'Guides')
    
    print('\n Writing 384 well IDT positions \n')

    write_384well_IDT_files (df_Primers_IDT, 'Primers')
    write_384well_IDT_files (df_Guides_IDT, 'Guides')
    
    return df_Primers_IDT, df_Guides_IDT

    
'''
The part below this point pertains to creating the 96-well or 
384-well formet for IDT order. The function for this purpose
have been exclsuiovely written down below. 
The writing to subfolder is a common function for whole script. 

'''


def generate_IDT_format(df):
    
    df  = df.reset_index(drop=True)
    df_96well_template = Template_96well()

    column_names = ["Well Position", "Name", "Sequence"]
    df_temp = pd.DataFrame(columns = column_names)

    total_DNA = len(df['Sequence'])
    #print(total_DNA, " <<-- This value must be less than 96")

    for index, row in df_96well_template.iterrows():
        Name = df['Name'].loc[index]
        Sequence = df['Sequence'].loc[index]
        position = df_96well_template['Well Position'].loc[index]

        list1 = [position, Name, Sequence]
        a_series = pd.Series(list1, index = df_temp.columns)
        df_temp = df_temp.append(a_series, ignore_index=True)

        if index+1 == total_DNA:
            break

    return df_temp

            
def write_96well_IDT_files (df_input, name):
     
    Plates = math.ceil(len(df_input['Name'])/96)

    for number in range(Plates):
        df_IDT = pd.DataFrame()
        df_pass = pd.DataFrame()
        start_ind = 96 * (number)
        end_ind = 96 * (number +1)

        if Plates == 1:
            df_IDT = generate_IDT_format(df_input)
        else:
            df_pass = (df_input[start_ind:end_ind])
            df_IDT = generate_IDT_format(df_pass)

        IDT_filename = name +'_EColi_96well_'+str(number+1)+'.csv'
        
        #write_to_subfolder(df_IDT, IDT_filename, 'IDT_Ordering')
        write_to_subfolder_excel(df_IDT, IDT_filename, 'IDT_Ordering')


def Template_96well():
    
    df_96well = pd.DataFrame()
    
    column_names = ["Well Position", "Name", "Sequence"]
    df_96well = pd.DataFrame(columns = column_names)
    
    for letter in ascii_uppercase:

        for i in range(12):
            Name = letter+str(i+1)

            list1 = [Name, '', '']
            a_series = pd.Series(list1, index = df_96well.columns)
            df_96well = df_96well.append(a_series, ignore_index=True)      

        if letter == 'H':
            break
                
    return df_96well
            
    
def generate_IDT_384well_format(df):
    
    df  = df.reset_index(drop=True)
    df_384well_template = well_plate_384()

    column_names = ["Well Position", "Name", "Sequence"]
    df_temp = pd.DataFrame(columns = column_names)

    total_DNA = len(df['Sequence'])
    #print(total_DNA, " <<-- This value must be less than 384 \n")
    

    for index, row in df_384well_template.iterrows():
        Name = df['Name'].loc[index]
        Sequence = df['Sequence'].loc[index]
        position = df_384well_template['Well Position'].loc[index]

        list1 = [position, Name, Sequence]
        a_series = pd.Series(list1, index = df_temp.columns)

        df_temp = df_temp.append(a_series, ignore_index=True)

        if index+1 == total_DNA:
            break

    return df_temp

            
def write_384well_IDT_files (df_input, name):
        
    Plates = math.ceil(len(df_input['Name'])/384)

    for number in range(Plates):
        df_IDT = pd.DataFrame()
        df_pass = pd.DataFrame()
        start_ind = 384 * (number)
        end_ind = 384 * (number +1)

        if Plates == 1:
            df_IDT = generate_IDT_384well_format(df_input)
        else:
            df_pass = (df_input[start_ind:end_ind])
            df_IDT = generate_IDT_384well_format(df_pass)

        IDT_filename = name +'_EColi_384well_'+str(number+1)+'.csv'
        #write_to_subfolder(df_IDT, IDT_filename, 'IDT_Ordering')
        write_to_subfolder_excel(df_IDT, IDT_filename, 'IDT_Ordering')

             
def well_plate_384():

    column_names = ["Well Position",  "Name", "Sequence"]
    df_384well_template = pd.DataFrame(columns = column_names)
    counter = 0

    for letter in ascii_uppercase:

        for i in range(24):
            counter = counter + 1
            Name = letter+str(i+1)

            list1 = [Name, '', '']
            a_series = pd.Series(list1, index = df_384well_template.columns)
            df_384well_template = df_384well_template.append(a_series, ignore_index=True)      

        if letter == 'P':
            break
    
    return df_384well_template
    
    
'''
This part below will compare all Original primers/guides against IDT order 
to verify that nothing is missing from IDT order. 
This is called from Create_Output() function.
'''


def compare_seqeunces(filenames, df_Primers_IDT, df_Guides_IDT):
    
    # Combine primers and guides into one Dataframe
    All_unique_df = df_Primers_IDT.append(df_Guides_IDT)
    
    df_not_found = pd.DataFrame()
    
    print('\n Comparing the original primer/guide files against IDT ordering file ')
    print(' in case of a mismtach, look for Failed_verification.csv file \n')
    
    for file in filenames:
        
        temp_df = pd.read_csv(file)
        print('\t Comparing file...    ', file)
        
        for index, row in temp_df.iterrows():
                Sequence = str(temp_df['Sequence'].loc[index])
                Primer_name = temp_df['Name'].loc[index]

                match = All_unique_df[All_unique_df['Sequence'].str.match(Sequence)]
                Match_counter = -1

                if match.empty:
                    df_not_found = df_not_found.append(temp_df[index:index+1])
                    print('Seqeunce Not FOUND!!!!!', Primer_name)

                elif not match.empty:
                    for index_I, row_I in match.iterrows():
                        Seq_match = str(match['Sequence'].loc[index_I])
                        if Sequence == Sequence:
                            Match_counter = 1

                if not match.empty and Match_counter != 1:
                    df_not_found = df_not_found.append(temp_df[index:index+1])
                    print('Sequence MISMATCH!!!!!')
   
    if df_not_found.empty:
        print("\n \t All primers and guides found in IDT order \n")
    else:
        print('\n Primers/Guides missing, check file :- Unqiue_varification.csv \n')
        write_to_subfolder(df_not_found, "Failed_verification.csv", 'IDT_Ordering')


    
if __name__ == '__main__':
    
    start_time = time.time()

    Create_Output()
    runtime = (time.time() - start_time)
    print("\nRuntime - - - {:.2f} seconds  - - -".format(runtime))

	 /Users/nilmani/Desktop/Python/High_GC_IDT 

File read:-   Primers_Guides_neb_1GC.csv
File read:-   Primers_Guides_neb_2GC.csv
File read:-   Primers_Guides_neb_3GC.csv
File read:-   Primers_Guides_neb_4GC.csv
File read:-   Primers_Guides_neb_5GC.csv

 These two must be equal:  
 	 Total number of primers and guides:-   132
 	 Number after combining all files:-   132 

File written  :-   Primers_guides_all_combined.csv
File written  :-   Primers_guides_unique.csv

 	 Need to add Template name and re-run the script 

File written  :-   Primers_for_IDT_order.csv

 	 Need to add Template name and re-run the script 

File written  :-   Guides_for_IDT_order.csv

  Writing 96 well IDT positions 

File written  :-   Primers_EColi_96well_1.xlsx
File written  :-   Guides_EColi_96well_1.xlsx

 Writing 384 well IDT positions 

File written  :-   Primers_EColi_384well_1.xlsx
File written  :-   Guides_EColi_384well_1.xlsx

 Comparing the original primer/guide files against IDT ordering file 
 in ca